In [11]:
import pymysql.cursors

# Подключение к базе данных
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='O1205lga+',
    database='ich_edit',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    connection.ping()
    print("Соединение установлено.")
except Exception as e:
    raise Exception(f"Ошибка соединения: {e}")

def get_table_data(table_name):
    """
    Получить данные из указанной таблицы.
    """
    try:
        with connection.cursor() as cursor:
            # Проверяем, существует ли таблица
            cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
            result = cursor.fetchone()
            if not result:
                return None

            cursor.execute(f"SELECT * FROM {table_name}")
            rows = cursor.fetchall()
            return rows
    except Exception as e:
        raise Exception(f"Error '{table_name}': {e}")

def get_user_purchases(user_name):
    """
    Получить все покупки пользователя по имени.
    """
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT id FROM Users WHERE name = %s", (user_name,))
            user = cursor.fetchone()
            if not user:
                raise ValueError(f"Пользователь с именем '{user_name}' не найден.")
            
            user_id = user['id']
            
            # Получаем список покупок пользователя
            cursor.execute("""
                SELECT Products.prod, Products.quantity 
                FROM Sales
                JOIN Products ON Sales.pid = Products.pid
                WHERE Sales.id = %s
            """, (user_id,))
            purchases = cursor.fetchall()
                        
            return purchases # Возвращаем список покупок
    except Exception as e:
        raise Exception(f"Ошибка при получении покупок пользователя '{user_name}': {e}")

while True:
    try:
        table_name = input("Введите название таблицы (Users, Products, Sales или '0' для завершения): ")
        
        if table_name == '0':
            print("Программа завершена.")
            break

        rows = get_table_data(table_name)
        if rows is None:
            print(f"Таблица '{table_name}' не найдена.")
        else:
            print(f"Содержимое таблицы '{table_name}':")
            for row in rows:
                print(row)
            
        # Если выбрана таблица Users,можно выбрать имя и показать покупки
        if table_name.lower() == 'users':
            print("\nДоступные имена пользователей:")
            user_names = [row['name'] for row in rows]
            print(", ".join(user_names))
            
            user_name = input("\nВыберите имя пользователя для отображения покупок: ")
            purchases = get_user_purchases(user_name)
            
            if purchases:
                print(f"\nПокупки пользователя '{user_name}':")
                for purchase in purchases:
                    print(f"Товар: {purchase['prod']}, Количество: {purchase['quantity']}")
            else:
                print(f"У пользователя '{user_name}' нет покупок.")
    except ValueError as ve:
        print(ve)
    except Exception as e:
        print(f"Произошла ошибка: {e}")


Соединение установлено.


Введите название таблицы (Users, Products, Sales или '0' для завершения):  Products


Содержимое таблицы 'Products':
{'pid': 1, 'prod': 'Laptop', 'quantity': 10}
{'pid': 2, 'prod': 'Mouse', 'quantity': 50}
{'pid': 3, 'prod': 'Keyboard', 'quantity': 30}


Введите название таблицы (Users, Products, Sales или '0' для завершения):  Users


Содержимое таблицы 'Users':
{'id': 1, 'name': 'Alice', 'age': 25}
{'id': 2, 'name': 'Bob', 'age': 30}
{'id': 3, 'name': 'Charlie', 'age': 22}

Доступные имена пользователей:
Alice, Bob, Charlie



Выберите имя пользователя для отображения покупок:  Alice



Покупки пользователя 'Alice':
Товар: Laptop, Количество: 10
Товар: Mouse, Количество: 50


Введите название таблицы (Users, Products, Sales или '0' для завершения):  0


Программа завершена.
